In [1]:
from new_implement_linear import *
import matplotlib.pyplot as plt

train_path="train.csv"
test_path="test.csv"
name="discrimination_submission.csv"

yb_train,data_train,ids_train=load_csv_data(train_path, sub_sample=False)

In [2]:
def algorithm_evaluation(data_train, yb_train, opt_model, cros_vali, degree, lambda_, gamma, outlier, method):
    
    seed = np.random.randint(cros_vali*100)
    
    print('step 1:Started main function!')
    data_train_copy = data_train.copy() 
    
    print('step 2:Feature augmentaion...')
    data_train = add_feature(data_train)            # add log and exp
    data_train = build_model_data(data_train, degree)  # add x^n
    
    print('step 3:Cross validation begins...')
    indices = build_k_indices(yb_train, cros_vali, seed) 
    corrections = cross_validation(data_train, yb_train, indices, opt_model, lambda_, outlier, method)
    
    print('step 7:End of Trainging\n')

    return np.mean(corrections)
    

In [3]:
# unified model choose
def train_mode(train_x, train_y, chosen_model, lambda_):
    if chosen_model   == 1:     
        w,loss=train_normal(train_y,train_x)
    
    elif chosen_model == 2:
        w,loss=train_ridge(train_y, train_x, lambda_)
    
    elif chosen_model == 3:
        w,loss=train_gradient(train_y, train_x)
    
    elif chosen_model == 4:
        w,loss=train_logistic(train_y, train_x, gamma)
        
    elif chosen_model == 5:
        w,loss=train_gradient_SGD(train_y, train_x)
                                   
#     elif chosen_model == 6:
        
    elif chosen_model == 7:
        class_1 = train_x[train_y == -1][:,1:train_x.shape[1]]
        class_2 = train_x[train_y == +1][:,1:train_x.shape[1]]
        
        w, loss = Fisher_classifier(class_1, class_2)
        
    return w, loss


# optimization models
def train_normal(yb_train, data_train):
    w,loss=least_squares(yb_train,data_train)
    return w,loss

def train_ridge(yb_train, data_train, lambda_):
    w,loss=ridge_regression(yb_train, data_train, lambda_)
    return w,loss

def train_gradient(yb_train, data_train, step_size):
    max_iters = 100
    gamma = step_size
    initial_w=np.zeros(len(data_train.T));
    w,loss=least_squares_GD(yb_train, data_train, initial_w, max_iters, gamma)
    return w,loss

def train_logistic(yb_train, data_train, step_size):
    initial_w=np.ones(len(data_train.T));
    max_iters=300;
    gamma=step_size;
    w,loss=logistic_regression(yb_train, data_train, initial_w, max_iters, gamma)
    return w,loss

def train_gradient_SGD(yb_train, data_train, step_size):
    batch_size=1;
    max_iters = 50
    gamma = step_size
    initial_w=np.zeros(len(data_train.T));
    w,loss=least_squares_SGD(yb_train, data_train, initial_w, max_iters, gamma,batch_size)
    return w,loss

# reserved def
 

def Fisher_classifier(class_1, class_2):
    # means
    m_1 = np.mean(class_1, axis = 0)
    m_2 = np.mean(class_2, axis = 0)
    
    # inner class1 dispersion
    S_1 = (class_1 - m_1).T.dot(class_1 - m_1)
    # inner class2 dispersion
    S_2 = (class_2 - m_2).T.dot(class_2 - m_2)
    # seperating level
    S_W = S_1 + S_2
    
    # perpendicular vector of the separating hyperplane
    w_star = np.linalg.inv(S_W.astype(float)).dot(m_2 - m_1)
    
    mapped_b = class_1.dot(w_star)
    m_t_b = mapped_b.mean()
    mapped_s = class_2.dot(w_star)
    m_t_s = mapped_s.mean()

    # intersection point of the separating hyperplane and the perpendicular vector 
    w0 = -0.5*(m_t_b + m_t_s)
    
    return np.insert(w_star, 0, w0), np.nan

##### Feature augmentation

In [4]:
def add_feature(data):
    r_data=data.copy()
    r_data=np.c_[r_data,np.exp(data[:,4])]
    r_data=np.c_[r_data,np.exp(data[:,6])]
    r_data=np.c_[r_data,np.exp(data[:,7])]

    r_data=np.c_[r_data,np.exp(data[:,12])]
    r_data=np.c_[r_data,np.exp(data[:,14])]
    r_data=np.c_[r_data,np.exp(data[:,15])]
    r_data=np.c_[r_data,np.exp(data[:,24])]
    r_data=np.c_[r_data,np.exp(data[:,18])]

    r_data=np.c_[r_data,np.cos(data[:,10])]
    r_data=np.c_[r_data,np.cos(data[:,24])]
    r_data=np.c_[r_data,np.cos(data[:,27])]

    r_data=np.c_[r_data,np.cos(data[:,11])]
    r_data=np.c_[r_data,np.log(data[:,9])]

    r_data=np.c_[r_data,np.log(data[:,10])]
    r_data=np.c_[r_data,np.log(data[:,13])]
    r_data=np.c_[r_data,np.log(data[:,19])]

    r_data=np.c_[r_data,data[:,range(7,10)]**5]

    return r_data

##### Data processing

In [5]:
def data_process(train_x, train_y, test_x, method, outlier):

    if method == 'no':
        print('No data processing is conducted')
    
    else:
        # replace outliers by nan
        if ~np.isnan(outlier):
            train_x[train_x == outlier] = np.nan
            test_x [test_x  == outlier] = np.nan
        else:
            print('No need to replace outlies by nan!\n')

        if method == 'mean':
            
            # only nan is replaced by mean value here
            train_x = fill_mean(train_x, train_y, 'train_data')
            test_x  = fill_mean(test_x , []     , 'test_data' )
            
        elif method == 'normalization':
            
            # To normalization, filling mean is necessary
            
            train_x = fill_mean(train_x, train_y, 'train_data')
            test_x  = fill_mean(test_x , []     , 'test_data' )
            
            train_x = normalization(train_x, train_y, 'train_data')      
            test_x  = normalization(test_x , []     , 'test_data' )
    
    
        elif method == 'drop':
            to_drop_train = np.isnan(train_x.min(axis = 0))
            to_drop_test  = np.isnan( test_x.min(axis = 0))
            to_drop = to_drop_train | to_drop_test
            
            train_x = train_x[:, ~to_drop]
            test_x  = test_x [:, ~to_drop]
            
        else:
            print('No such processing method! Please check your input!')
            return -1
        
    return train_x, test_x  


def fill_mean(data, label, category):
    
    if category == 'train_data':
        class_1 = data[label == 1 ]
        class_2 = data[label == -1]
        
        mean_1 = np.nanmean(class_1,axis = 0)
        mean_2 = np.nanmean(class_2,axis = 0)
        
        for ct_i in range(class_1.shape[1]):
            class_1[ np.isnan(class_1[:,ct_i]), ct_i] = mean_1[ct_i]
            class_2[ np.isnan(class_2[:,ct_i]), ct_i] = mean_2[ct_i]
        
        data[label == 1 ] = class_1
        data[label == -1] = class_2
        
    elif category == 'test_data':
        mean = np.nanmean(data,axis = 0)     
        for ct_i in range(data.shape[1]):
            data[ np.isnan(data[:,ct_i]), ct_i] = mean[ct_i]
            
    else:
        print('No such data category! Please check your input!')
        
    return data


def normalization(data, label, category):
    
    eps = 1e-2
    
    if category == 'train_data':
        class_1 = data[label == 1, 1:data.shape[1] ]
        class_2 = data[label == -1, 1:data.shape[1]]
        
        class_1 = (class_1 - np.nanmin(class_1,axis=0))/(np.nanmax(class_1,axis=0) - np.nanmin(class_1,axis=0) + eps)
        class_2 = (class_2 - np.nanmin(class_2,axis=0))/(np.nanmax(class_2,axis=0) - np.nanmin(class_2,axis=0) + eps)   
        
        data[label == 1, 1:data.shape[1] ] = class_1
        data[label == -1, 1:data.shape[1]] = class_2
                
    elif category == 'test_data':
        data_temp = data[:, 1:data.shape[1]].copy()
        data_temp = (data_temp - np.nanmin(data_temp,axis=0))/(np.nanmax(data_temp,axis=0) - np.nanmin(data_temp,axis=0) + eps)

        data[:, 1:data.shape[1]] = data_temp.copy()
    else:
        print('No such data category! Please check your input!')
        
    return data
    

##### Cross validation

In [6]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                for k in range(k_fold)]
    return np.array(k_indices)


def cross_validation(feature, lable, indices, opt_model, lambda_, outlier, method):
    '''cross validation to evaluate performance of a given mode.
    
       feature (ndarray)  : feature vectors
       lable   (array)    : lable of every row of feature
       indices            : grouped indices of the dataset
       opt_model(int)     : chosen classification model 
    '''
    
    iter_times = indices.shape[0]
    correction = []
    
    for ct_iter in range(iter_times):
        # produce training data
        train_x = feature[indices[ct_iter]]
        train_y = lable[indices[ct_iter]]
        
        # produce testing data
        test_indices = indices[np.arange(indices.shape[0])!=ct_iter]
        test_indices = test_indices.ravel()
        test_x = feature[test_indices]
        test_y = lable[test_indices]
        
        print('  -> Data processing:', method)
        train_x, test_x = data_process(train_x, train_y, test_x, method, outlier)
        
        w, loss = train_mode(train_x, train_y, opt_model, lambda_)
        prediction = predict_labels(w, test_x)
            
        correction.append( sum(prediction == test_y.astype(float))/test_y.shape[0] )
            
        print('          --** Accuracy of cross validation', ct_iter, 'is:', correction[ct_iter])
    return correction

In [8]:
# main function here

#  algrithm=1  Least Squrare
#  algrithm=2  Ridge Regression
#  algrithm=3   其他的暂时有bug
#  ...
#  algrithm=7  Fisher Linear Discriminant

outlier = -999
method =  'drop' #  all methods: 'mean', 'normalization', 'drop', 'no'

algrithm = 2
cros_slice = 5
degree = 1
lambda_= np.logspace(-10, 1, num = 20, endpoint=True) 
gamma = 0.1 # step_size

mean_correct = np.zeros(len(lambda_))

for ct_i in range(len(lambda_)):
    mean_correct[ct_i] = algorithm_evaluation(data_train    , yb_train, 
                                              algrithm      , cros_slice, 
                                              degree        , lambda_[ct_i], 
                                              gamma         , outlier, 
                                              method)


step 1:Started main function!
step 2:Feature augmentaion...
step 3:Cross validation begins...
  -> Data processing: drop
          --** Accuracy of cross validation 0 is: 0.77111
  -> Data processing: drop
          --** Accuracy of cross validation 1 is: 0.76952
  -> Data processing: drop
          --** Accuracy of cross validation 2 is: 0.769815
  -> Data processing: drop
          --** Accuracy of cross validation 3 is: 0.769315
  -> Data processing: drop
          --** Accuracy of cross validation 4 is: 0.76956
step 7:End of Trainging

step 1:Started main function!
step 2:Feature augmentaion...
step 3:Cross validation begins...
  -> Data processing: drop
          --** Accuracy of cross validation 0 is: 0.77075
  -> Data processing: drop
          --** Accuracy of cross validation 1 is: 0.768585
  -> Data processing: drop
          --** Accuracy of cross validation 2 is: 0.771775
  -> Data processing: drop
          --** Accuracy of cross validation 3 is: 0.768255
  -> Data process

step 3:Cross validation begins...
  -> Data processing: drop
          --** Accuracy of cross validation 0 is: 0.754185
  -> Data processing: drop
          --** Accuracy of cross validation 1 is: 0.755435
  -> Data processing: drop
          --** Accuracy of cross validation 2 is: 0.7558
  -> Data processing: drop
          --** Accuracy of cross validation 3 is: 0.754935
  -> Data processing: drop
          --** Accuracy of cross validation 4 is: 0.75541
step 7:End of Trainging

step 1:Started main function!
step 2:Feature augmentaion...
step 3:Cross validation begins...
  -> Data processing: drop
          --** Accuracy of cross validation 0 is: 0.751525
  -> Data processing: drop
          --** Accuracy of cross validation 1 is: 0.750105
  -> Data processing: drop
          --** Accuracy of cross validation 2 is: 0.749605
  -> Data processing: drop
          --** Accuracy of cross validation 3 is: 0.75066
  -> Data processing: drop
          --** Accuracy of cross validation 4 is: 

In [ ]:
mean_correct

In [ ]:
fig = plt.figure()

plt.xscale('log')
plt.plot(lambda_, mean_correct)

plt.xlabel('Ridge regression coefficients ')
plt.ylabel('Accuracy')
plt.legend(['coefficiency'])
plt.show()

fig.savefig('Ridge regression coefficiency.png')

In [ ]:
fig = plt.figure()

plt.plot(degree, mean_correct)
plt.xlabel('Polynomial feature augmentation')
plt.ylabel('Accuracy')
plt.legend(['Accuracy'])
plt.show()

fig.savefig('Least square with different dgree.png')